In [1]:
import os
import pandas as pd
from PIL import Image
from torch.utils.data import DataLoader
from torchvision import transforms
import torch
import torch.nn as nn
import numpy as np
import snntorch as snn
from snntorch import functional as SF

In [ ]:

class ChirpletDataset():
    """ loading the chirplet plots and labels from the corresponding csv file
        each csv file contains the corresponding label for each chirplet plot on the 4th or last column
        after extracting the labels, converting the strings to int label as 0 1 2 
        because the tensor type only contains numeric values 
    """
    def __init__(self, img_dir, label_dir, transform=None):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.transform = transform
        
        # filenames without extension
        self.filenames = [os.path.splitext(f)[0] for f in os.listdir(img_dir) if f.endswith('.png')]
        
        # mapping of unique labels to integers
        self.label_mapping = self._create_label_mapping()

    def _create_label_mapping(self):
        unique_labels = set()
        for filename in self.filenames:
            label_path = os.path.join(self.label_dir, filename + '.csv')
            label_df = pd.read_csv(label_path, header=None)
            label = label_df.iloc[0, -1]
            unique_labels.add(label)
        return {label: i for i, label in enumerate(sorted(unique_labels))}

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img_name = self.filenames[idx]
        img_path = os.path.join(self.img_dir, img_name + '.png')
        label_path = os.path.join(self.label_dir, img_name + '.csv')
        
        # Convert image to grayscale
        image = Image.open(img_path).convert('L')

        label_df = pd.read_csv(label_path, header=None)
        label = label_df.iloc[0, -1]  

        label_int = self.label_mapping[label]

        label_tensor = torch.tensor(label_int, dtype=torch.long)

        if self.transform:
            image = self.transform(image)
        
        return image, label_tensor

# Training Parameters
batch_size = 3
epochs = 10
trainDataPath = './data/chirplet_plots/train/'
testDataPath = './data/chirplet_plots/test/'
labelPath = './data/audio_dataset/'

# Data Preprocessing
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),  # convert the image to tensor
    transforms.Normalize((0.5,), (0.5,))  # (mean, std) for grayscale images
])

# initialize dataset and DataLoader
train_dataset = ChirpletDataset(img_dir=trainDataPath, label_dir=labelPath, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = ChirpletDataset(img_dir=testDataPath, label_dir=labelPath, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

# training dataloader
for images, labels in train_loader:
    print(f"shapes -> images: {images.shape}, labels: {labels.shape}")
    print(f"label values in a single batch: {labels}")
    break

print(f"total labels: {len(labels)} \t label type: {type(labels)}")
print(f"total batches: {len(train_loader)}")
print(f"unique classes: {len(train_dataset.label_mapping)}")

# testing dataloader
for images, labels in test_loader:
    print(f"\nshapes -> images: {images.shape}")
    print(f"label values in a single batch: {labels}")
    break

print(f"total batches: {len(test_loader)}")
print(f"unique classes: {len(test_dataset.label_mapping)}")

In [6]:
def save_to_tensors(train_loader, test_loader):
    """ saving as tensors rather than numpy format to avoid numpy to tensor conversion
        ...
    """
    # empty lists to hold the training images and labels
    train_images = []
    train_labels = []

    # loop through each batch of training data
    for images, labels in train_loader:
        # Append the tensors directly without converting to numpy
        train_images.append(images)
        train_labels.append(labels)

    # create empty lists for test images and labels
    test_images = []
    test_labels = []

    # loop through each batch of testing data
    for images, labels in test_loader:
        # Append the tensors directly
        test_images.append(images)
        test_labels.append(labels)

    # concatenate all training and test tensors
    train_images_tensor = torch.cat(train_images, dim=0)
    train_labels_tensor = torch.cat(train_labels, dim=0)
    test_images_tensor = torch.cat(test_images, dim=0)
    test_labels_tensor = torch.cat(test_labels, dim=0)

    # save the tensors using torch.save()
    torch.save(train_images_tensor, './data/dump/train_images.pt')
    torch.save(train_labels_tensor, './data/dump/train_labels.pt')
    torch.save(test_images_tensor, './data/dump/test_images.pt')
    torch.save(test_labels_tensor, './data/dump/test_labels.pt')
# Call the function to save the preprocessed data as tensors
save_to_tensors(train_loader, test_loader)

In [7]:
# retrieving the numpy files to input in the model and converting from numpy to tensor
train_images_tensor = torch.load('./data/dump/train_images.pt')
train_labels_tensor = torch.load('./data/dump/train_labels.pt')
test_images_tensor = torch.load('./data/dump/test_images.pt')
test_labels_tensor = torch.load('./data/dump/test_labels.pt')


In [36]:

# # Model parameters
# # Model config hyperparameters
# batch_size = 3
# num_steps = 25
# num_epochs = 5 
# alpha = 0.9
# beta = 0.85
# # using gpu
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# # SNN Network
# class Net(nn.Module):
#     def __init__(self, num_inputs, num_hidden, num_outputs):
#         super().__init__()

#         # Initialize layers
#         self.fc1 = nn.Linear(num_inputs, num_hidden)
#         self.lif1 = snn.Leaky(beta=beta)
#         self.fc2 = nn.Linear(num_hidden, num_outputs)
#         self.lif2 = snn.Leaky(beta=beta)

#     def forward(self, x):
#         mem1 = self.lif1.init_leaky()
#         mem2 = self.lif2.init_leaky()

#         spk2_rec = []
#         mem2_rec = []

#         for _ in range(num_steps):
#             cur1 = self.fc1(x)
#             spk1, mem1 = self.lif1(cur1, mem1)
#             cur2 = self.fc2(spk1)
#             spk2, mem2 = self.lif2(cur2, mem2)
#             spk2_rec.append(spk2)
#             mem2_rec.append(mem2)

#         return torch.stack(spk2_rec, dim=0), torch.stack(mem2_rec, dim=0)


# num_inputs = 28 * 28  
# num_hidden = 1000
# num_outputs = len(train_dataset.label_mapping)  

# # Initialize Network
# net = Net(num_inputs, num_hidden, num_outputs).to(device)
# # Initialize Optimizer
# optimizer = torch.optim.Adam(net.parameters(), lr=1e-3, betas=(0.9, 0.999))
# # Define the loss function
# loss_fn = SF.ce_rate_loss()

# # Training Loop
# for epoch in range(num_epochs):
#     total_loss = 0
#     for i, (images, labels) in enumerate(train_loader):
#         images = images.view(-1, 28*28).to(device)
#         labels = labels.to(device)

#         # Forward pass
#         spk_rec, mem_rec = net(images)
        
#         loss = torch.zeros((1), dtype=torch.float, device=device)
#         for step in range(num_steps):
#             loss += loss_fn(mem_rec[step], labels)

#         # Gradient calculation + weight update
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item()

#         # Print the results
#         if (i+1) % 100 == 0:
#             avg_loss = total_loss / 100
#             print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Avg Loss: {avg_loss:.4f}')
#             total_loss = 0

# # Evaluation function
# def test_accuracy(data_loader, net, num_steps):
#     with torch.no_grad():
#         total = 0
#         acc = 0
#         net.eval()
        
#         for data, targets in data_loader:
#             data = data.view(-1, 28*28).to(device)
#             targets = targets.to(device)
#             spk_rec, _ = net(data)
            
#             acc += SF.accuracy_rate(spk_rec, targets) * spk_rec.size(1)
#             total += spk_rec.size(1)

#     return acc/total

# # Test the model
# test_acc = test_accuracy(train_loader, net, num_steps)
# print(f"Test Accuracy: {test_acc * 100:.2f}%")

Test Accuracy: 50.00%


In [52]:
# Network Architecture
num_inputs = 28 * 28
num_hidden = 1000
num_outputs = 10

# Temporal Dynamics
num_steps = 25
beta = 0.95
dtype = torch.float

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# Define Network
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        # Initialize layers
        self.fc1 = nn.Linear(num_inputs, num_hidden)
        self.lif1 = snn.Leaky(beta=beta)
        self.fc2 = nn.Linear(num_hidden, num_outputs)
        self.lif2 = snn.Leaky(beta=beta)

    def forward(self, x):

        # Initialize hidden states at t=0
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()

        # Record the final layer
        spk2_rec = []
        mem2_rec = []

        for step in range(num_steps):
            cur1 = self.fc1(x)
            spk1, mem1 = self.lif1(cur1, mem1)
            cur2 = self.fc2(spk1)
            spk2, mem2 = self.lif2(cur2, mem2)
            spk2_rec.append(spk2)
            mem2_rec.append(mem2)

        return torch.stack(spk2_rec, dim=0), torch.stack(mem2_rec, dim=0)


# Load the network onto CUDA if available
net = Net().to(device)
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=5e-4, betas=(0.9, 0.999))
data, targets = next(iter(train_loader))
data = data.to(device)
targets = targets.to(device)
spk_rec, mem_rec = net(data.view(batch_size, -1))
# initialize the total loss value
loss_val = torch.zeros((1), dtype=dtype, device=device)

# sum loss at every step
for step in range(num_steps):
    loss_val += loss(mem_rec[step], targets)

# clear previously stored gradients
optimizer.zero_grad()

# calculate the gradients
loss_val.backward()

# weight update
optimizer.step()
# calculate new network outputs using the same data
spk_rec, mem_rec = net(data.view(batch_size, -1))

# initialize the total loss value
loss_val = torch.zeros((1), dtype=dtype, device=device)

# sum loss at every step
for step in range(num_steps):
    loss_val += loss(mem_rec[step], targets)
num_epochs = 1
loss_hist = []
test_loss_hist = []
counter = 0


def print_batch_accuracy(data, targets, train=False):
    output, _ = net(data.view(batch_size, -1))
    _, idx = output.sum(dim=0).max(1)
    acc = np.mean((targets == idx).detach().cpu().numpy())

    if train:
        print(f"Train set accuracy for a single minibatch: {acc*100:.2f}%")
    else:
        print(f"Test set accuracy for a single minibatch: {acc*100:.2f}%")


def train_printer():
    print(f"Epoch {epoch}, Iteration {iter_counter}")
    print(f"Train Set Loss: {loss_hist[counter]:.2f}")
    print(f"Test Set Loss: {test_loss_hist[counter]:.2f}")
    print_batch_accuracy(data, targets, train=True)
    print_batch_accuracy(test_data, test_targets, train=False)
    print("\n")


# Outer training loop
for epoch in range(num_epochs):
    iter_counter = 0
    train_batch = iter(train_loader)

    # Minibatch training loop
    for data, targets in train_batch:
        data = data.to(device)
        targets = targets.to(device)

        # forward pass
        net.train()
        spk_rec, mem_rec = net(data.view(batch_size, -1))

        # initialize the loss & sum over time
        loss_val = torch.zeros((1), dtype=dtype, device=device)
        for step in range(num_steps):
            loss_val += loss(mem_rec[step], targets)

        # Gradient calculation + weight update
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()

        # Store loss history for future plotting
        loss_hist.append(loss_val.item())

        # Test set
        with torch.no_grad():
            net.eval()
            test_data, test_targets = next(iter(test_loader))
            test_data = test_data.to(device)
            test_targets = test_targets.to(device)

            # Test set forward pass
            test_spk, test_mem = net(test_data.view(batch_size, -1))

            # Test set loss
            test_loss = torch.zeros((1), dtype=dtype, device=device)
            for step in range(num_steps):
                test_loss += loss(test_mem[step], test_targets)
            test_loss_hist.append(test_loss.item())

            # Print train/test loss/accuracy
            if counter % 50 == 0:
                train_printer()
            counter += 1
            iter_counter += 1

RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# Training Parameters
batch_size = 32
num_steps = 25
num_epochs = 10
beta = 0.85
learning_rate = 1e-3

trainDataPath = './data/chirplet_plots/train/'
testDataPath = './data/chirplet_plots/test/'
labelPath = './data/audio_dataset/'

# Data Preprocessing
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Initialize datasets and DataLoaders
train_dataset = ChirpletDataset(img_dir=trainDataPath, label_dir=labelPath, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = ChirpletDataset(img_dir=testDataPath, label_dir=labelPath, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Model parameters
num_inputs = 28 * 28
num_hidden = 1000
num_outputs = len(train_dataset.label_mapping)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize Network
net = Net(num_inputs, num_hidden, num_outputs, beta).to(device)

# Initialize Optimizer
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, betas=(0.9, 0.999))

# Define the loss function
loss_fn = SF.ce_rate_loss()

# Training Loop
for epoch in range(num_epochs):
    total_loss = 0
    net.train()
    for i, (images, labels) in enumerate(train_loader):
        print(f"Batch {i+1}:")
        print(f"Images shape: {images.shape}")
        print(f"Labels shape: {labels.shape}")
        print(f"Unique labels: {torch.unique(labels)}")

        images = images.view(-1, 28*28).to(device)
        labels = labels.to(device)

        # Forward pass
        spk_rec, mem_rec = net(images, num_steps)
        
        print(f"spk_rec shape: {spk_rec.shape}")
        print(f"mem_rec shape: {mem_rec.shape}")

        loss = torch.zeros((1), dtype=torch.float, device=device)
        for step in range(num_steps):
            print(f"Step {step}:")
            print(f"mem_rec[step] shape: {mem_rec[step].shape}")
            print(f"labels shape: {labels.shape}")
            loss += loss_fn(mem_rec[step], labels)

        # Gradient calculation + weight update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Print the results
        if (i+1) % 100 == 0:
            avg_loss = total_loss / 100
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Avg Loss: {avg_loss:.4f}')
            total_loss = 0

        # Break after the first batch to avoid flooding the output
        break

    # Break after the first epoch
    break

    # Test the model after each epoch
    train_acc = test_accuracy(train_loader, net, num_steps, device)
    test_acc = test_accuracy(test_loader, net, num_steps, device)
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Accuracy: {train_acc * 100:.2f}%")
    print(f"Test Accuracy: {test_acc * 100:.2f}%")